In [1]:
import requests
year = 2022
day = 19

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open('cookie.txt', 'r') as file:
    session = file.read()

headers = {'cookie': f"session={session}"}

with open(f'day_{day}_input.txt', 'wb') as file:
    file.write(requests.get(url, headers=headers).content)
    
import os
if os.path.exists(f"day_{day}_test.txt"):
    test = [x.strip() for x in open(f'day_{day}_test.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
if os.path.exists(f"day_{day}_test_2.txt"):
    test_2 = [x.strip() for x in open(f'day_{day}_test_2.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
lines = [x.strip() for x in open(f'day_{day}_input.txt', 'r')]
for line in lines[:10]:
    print(line.strip())

---
---
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 11 clay. Each geode robot costs 4 ore and 8 obsidian.
Blueprint 2: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 2 ore and 16 clay. Each geode robot costs 4 ore and 16 obsidian.
Blueprint 3: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 18 clay. Each geode robot costs 2 ore and 11 obsidian.
Blueprint 4: Each ore robot costs 2 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 17 clay. Each geode robot costs 2 ore and 10 obsidian.
Blueprint 5: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 10 clay. Each geode robot costs 4 ore and 10 obsidian.
Blueprint 6: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 20 obsidian.
Blueprint 7: Each ore robot 

In [2]:
import re
from collections import deque

run_test = False

if run_test:
    case = test
else:
    case = lines

pattern = "Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian."

ORE = 0
CLAY = 1
OBS = 2
GEO = 3


def solve(ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs):
    
    first_state = (1,0,0,0,0,0,0,0,0)
    
    highest_geodes = 0
    
    Q = deque([first_state])
    
    states = set()
    
    while Q:
        state = Q.popleft()
        #print(state)
        if state in states:
            continue
        states.add(state)
        
        current_robots = state[:4]
        current_resources = state[4:8]
        minutes = state[8]
        #print(state,current_robots, current_resources, time)
        
        if current_resources[GEO] > highest_geodes:
            print(state)
            highest_geodes = current_resources[GEO]
        
        if minutes == STOP:
            #print(state)
            continue
            
        if len(states) % 1000000 == 0:
            print(minutes,highest_geodes,len(states))
            
#         for s in [s for s in states if s[8] == minutes and not state == s]:
#             state_is_better = True
#             if all([s[i] >= state[i] for i in range(8)]):
#                 #print('state sucks')
#                 continue
        
        
        
        max_ore_required = (STOP-minutes-1)*(ore_ore + clay_ore + obs_ore + geo_ore)
        max_clay_required = (STOP-minutes-1)*(obs_clay)
        max_obs_required = (STOP-minutes-1)*(geo_obs)

        new_states = list()

        
        # make geode
        if current_resources[ORE] >= geo_ore and current_resources[OBS] >= geo_obs:
            new_resources = list(current_resources)
            new_robots = [0,0,0,1]
            new_resources[OBS] -= geo_obs
            new_resources[ORE] -= geo_ore
            new_states.append(new_robots+new_resources+[minutes+1])
        else:
            # make obs
            if current_resources[ORE] >= obs_ore and current_resources[CLAY] >= obs_clay and current_resources[OBS] < max_obs_required:
                new_resources = list(current_resources)
                new_robots = [0,0,1,0]
                new_resources[ORE] -= obs_ore
                new_resources[CLAY] -= obs_clay
                new_states.append(new_robots+new_resources+[minutes+1])
                
            # make clay
            if current_resources[ORE] >= clay_ore and current_resources[CLAY] < max_clay_required:
                new_resources = list(current_resources)
                new_robots = [0,1,0,0]
                new_resources[ORE] -= clay_ore
                new_states.append(new_robots+new_resources+[minutes+1])

            # make ore
            if current_resources[ORE] >= ore_ore and current_resources[ORE] < max_ore_required:
                new_resources = list(current_resources)
                new_robots = [1,0,0,0]
                new_resources[ORE] -= ore_ore
                new_states.append(new_robots+new_resources+[minutes+1])

            
            new_states.append([0,0,0,0]+list(current_resources)+[minutes+1])


        for new_state in new_states:
            for i in range(4):
                new_state[i+4] += current_robots[i]
                new_state[i] += current_robots[i]

            new_state[4+ORE] = min(max_ore_required,new_state[4+ORE])
            new_state[4+CLAY] = min(max_clay_required,new_state[4+CLAY])
            new_state[4+OBS] = min(max_obs_required,new_state[4+OBS])
                
        #print(len(new_states))
        #print(new_states)
        
        Q.extend([tuple(ns) for ns in new_states])
        #print(len(Q))
    return highest_geodes
            
    
STOP = 24

list_of_bests = []

for line in case:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    list_of_bests.append(id_ * solve(ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs))

print(sum(list_of_bests))


(1, 3, 2, 1, 2, 15, 4, 1, 22)
(1, 3, 3, 1, 1, 7, 6, 2, 23)
(1, 3, 3, 1, 0, 0, 0, 3, 24)
(4, 11, 3, 1, 0, 0, 0, 1, 24)
19 0 1000000
(2, 10, 4, 1, 3, 23, 6, 1, 20)
20 1 2000000
(2, 10, 5, 1, 3, 16, 10, 2, 21)
22 2 3000000
(2, 10, 5, 2, 3, 26, 5, 3, 22)
(2, 10, 6, 2, 3, 17, 10, 4, 22)
(2, 10, 6, 2, 3, 17, 10, 5, 23)
(2, 10, 6, 3, 3, 17, 6, 6, 23)
(2, 10, 6, 3, 0, 0, 0, 7, 24)
(2, 10, 6, 3, 0, 0, 0, 9, 24)
(1, 3, 3, 1, 2, 12, 8, 1, 21)
(1, 3, 4, 1, 1, 5, 11, 2, 22)
22 2 1000000
(1, 3, 4, 1, 2, 8, 10, 3, 23)
(1, 3, 4, 1, 0, 0, 0, 4, 24)
(2, 6, 4, 2, 0, 0, 0, 5, 24)
18 0 1000000
19 0 2000000
(3, 7, 7, 1, 3, 16, 12, 1, 20)
20 1 3000000
(3, 7, 8, 1, 3, 15, 19, 2, 21)
21 2 4000000
(3, 7, 9, 1, 3, 14, 27, 3, 22)
(3, 7, 9, 2, 3, 8, 16, 4, 23)
(4, 7, 8, 2, 0, 0, 0, 5, 24)
(3, 7, 9, 2, 0, 0, 0, 6, 24)
23 0 1000000
(3, 5, 6, 1, 4, 9, 12, 1, 20)
20 1 1000000
(3, 5, 6, 2, 5, 14, 8, 2, 21)
(3, 5, 7, 2, 5, 12, 14, 4, 22)
(3, 5, 7, 3, 6, 6, 10, 6, 23)
(3, 5, 7, 4, 0, 0, 0, 9, 24)
20 0 1000000
(4, 6, 6, 1

In [3]:
# Don't consider building a robot if you know the current set of robots will already provide more resources of that type
# than you'll ever need. If you've already got 90 clay by turn 20 you probably don't need to go from 10 to 11 clay robots.

STOP = 32
list_of_32_bests = []
for line in case[:3]:
    groups = re.findall(pattern,line)
    id_, ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs = [int(x) for x in groups[0]]
    list_of_32_bests.append(solve(ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs))
    
print(list_of_32_bests[0]*list_of_32_bests[1]*list_of_32_bests[2])

(1, 3, 2, 1, 2, 15, 4, 1, 22)
(1, 3, 3, 1, 1, 7, 6, 2, 23)
23 2 1000000
(1, 3, 3, 1, 2, 10, 9, 3, 24)
24 3 2000000
24 3 3000000
(1, 3, 3, 1, 3, 13, 12, 4, 25)
25 4 4000000
25 4 5000000
25 4 6000000
(1, 3, 4, 1, 2, 5, 15, 5, 26)
(1, 3, 2, 2, 2, 27, 4, 6, 26)
26 6 7000000
26 6 8000000
26 6 9000000
26 6 10000000
27 6 11000000
(1, 3, 3, 2, 1, 19, 6, 8, 27)
27 8 12000000
27 8 13000000
(2, 5, 5, 3, 3, 19, 8, 9, 27)
27 9 14000000
27 9 15000000
27 9 16000000
27 9 17000000
28 9 18000000
(1, 3, 3, 2, 2, 22, 9, 10, 28)
28 10 19000000
28 10 20000000
28 10 21000000
(2, 5, 6, 3, 3, 13, 15, 11, 28)
(2, 5, 6, 3, 3, 13, 13, 12, 28)
28 12 22000000
28 12 23000000
28 12 24000000
29 12 25000000
29 12 26000000
29 12 27000000
29 12 28000000
(3, 5, 6, 4, 5, 18, 11, 14, 29)
(2, 5, 7, 3, 3, 7, 19, 15, 29)
(2, 5, 5, 4, 3, 29, 8, 16, 29)
29 16 29000000
30 16 30000000
30 16 31000000
(3, 5, 6, 5, 4, 22, 9, 18, 30)
(2, 5, 6, 4, 3, 22, 15, 19, 30)
(2, 5, 6, 4, 3, 22, 13, 20, 30)
31 20 32000000
(3, 5, 6, 6, 3, 11, 7, 